# 5. 셀레니움 활용 웹브라우저 자동화

### (1) 라이브러리 선언 및 드라이버 설정

In [72]:
!pip show selenium

Name: selenium
Version: 4.23.1
Summary: Official Python bindings for Selenium WebDriver
Home-page: https://www.selenium.dev
Author: 
Author-email: 
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: certifi, trio, trio-websocket, typing_extensions, urllib3, websocket-client
Required-by: 


In [73]:
# Selenium과 webdriver-manager를 사용한 Chrome 브라우저 실행 코드
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import bs4

In [74]:
!pip show webdriver-manager

Name: webdriver-manager
Version: 4.0.2
Summary: Library provides the way to automatically manage drivers for different browsers
Home-page: https://github.com/SergeyPirogov/webdriver_manager
Author: Sergey Pirogov
Author-email: automationremarks@gmail.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: packaging, python-dotenv, requests
Required-by: 


In [75]:
# Colab에 필요한 패키지 설치
!pip install selenium
!pip install webdriver-manager

In [76]:
# 2. Chrome 브라우저 설치
!apt-get update  # 패키지 목록 업데이트
!apt-get install -y wget unzip  # wget과 unzip 패키지 설치
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -  # Google의 리눅스 서명 키 추가
!sh -c 'echo "deb [arch=amd64] https://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list'  # Chrome 브라우저를 다운로드할 수 있도록 저장소 추가
!apt-get update  # 패키지 목록 다시 업데이트
!apt-get install -y google-chrome-stable  # 최신 안정 버전의 Chrome 브라우저 설치
!apt-get install -y fonts-nanum # 한글폰트 설치

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:14 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy/main amd64 Packages [50.1 kB]
Fetched 335 kB in 2s (174 

### 코랩인 경우 해당 셀내 ★ 주석해제 필요

In [77]:
# Selenium과 webdriver-manager를 사용한 Chrome 브라우저 실행 코드
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import bs4

# WebDriver 설치 및 브라우저 실행 설정
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # ★헤드리스 모드로 실행 (브라우저 창을 표시하지 않음)
options.add_argument("window-size=1920x1080") # ★헤드리스 모드로 실행 (명시적으로 표기)
options.add_argument("--no-sandbox")  # 샌드박스 모드 비활성화 (가상 환경에서 안정적인 실행을 위해)
options.add_argument("--disable-dev-shm-usage")  # /dev/shm 사용 비활성화 (메모리 부족 방지)
options.add_argument("--disable-gpu")  # GPU 가속 비활성화 (필요시 사용)
options.add_argument("start-maximized")  # 브라우저 창 최대화
options.add_argument("disable-infobars")  # 정보 표시줄 비활성화
options.add_argument("--disable-extensions")  # 브라우저 확장 프로그램 비활성화
# user-agent 값 설정
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
options.add_argument(f"user-agent={user_agent}")

# WebDriver로 Chrome 브라우저 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
# 웹페이지 파싱 될때까지 최대 3초 기다림
driver.implicitly_wait(3)

In [78]:
# WebDriver로 Chrome 브라우저 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [79]:
driver.current_url

'data:,'

In [80]:
driver.save_screenshot("here.jpg")

/usr/local/lib/python3.10/dist-packages/selenium/webdriver/remote/webdriver.py:829: UserWarning: name used for saved screenshot does not match file type. It should end with a `.png` extension
  return self.get_screenshot_as_file(filename)


True

### 1. 웹페이지 이동 + 스크린샷

In [81]:
pwd

'/content'

In [82]:
baseUrl = "https://www.sparkkorea.com"
driver.get( baseUrl )
# 현재 위치하고 있는 사이트의 주소를 반환
driver.current_url
# 현재 드라이버의 페이지를 스크린샷
driver.save_screenshot("here.jpg")

True

### 2. 버튼 클릭 및 데이터 수집까지 쭉 bs4

In [83]:
baseUrl = "https://www.sparkkorea.com"
driver.get( baseUrl )

In [84]:
driver.current_url

'https://sparkkorea.com/'

In [85]:
# xpath url 탐색 (단, xpath, fullxpath)
quizBtnXpath = '//*[@id="menu-item-382"]/a'
# 드라이버에게 버튼찾아라 (못찾으면 웹페이지 확인)
quizBtnEle = driver.find_element(By.XPATH, quizBtnXpath)
# 해당 요소를 클릭
quizBtnEle.click()

In [86]:
resp = requests.get("https://www.sparkkorea.com/퀴즈")
html = resp.text
bs = bs4.BeautifulSoup(html, "html.parser")

In [87]:
html = driver.page_source
bs = bs4.BeautifulSoup(html, "html.parser")

In [88]:
 div = bs.find(name = "div", attrs={ "class":"class_spark_quiz"})

In [89]:
aTags = div.findAll(name = "a")
aTags

[<a href="https://forms.gle/Fw49w9GhWQChDcZm7"> 6/13 Spark 심화과정 </a>,
 <a href="https://forms.gle/G4TcXm3fKuHLHA6D6"> 6/13 Spark 기본과정 </a>,
 <a href="https://forms.gle/M8gr1kC2ubA3UDVp8"> 6/18 Spark [MAP_FILTER] </a>,
 <a href="https://forms.gle/h8w5mZ4MNaPLCPbi6"> 6/18 Spark GroupBy 심화 </a>,
 <a href="https://forms.gle/q5yL6QHfueDLM5w27"> 6/25 Spark RDD 실전 분석 </a>,
 <a href="https://forms.gle/Gxb4y6LfVYiaLu4M7"> 6/27 Spark RDD 실전 분석2 </a>]

### 3. 구글 접속 후 텍스트 입력하기

In [90]:
baseUrl = "https://www.google.com"

In [91]:
# google.com 에 접속
driver.get(baseUrl)
# 서치 바 탐색
gSearchXpath = '//*[@id="APjFqb"]'
gSearchEle = driver.find_element(By.XPATH, gSearchXpath)
# 키 입력
inputText = "selenium"
gSearchEle.send_keys(inputText)
driver.save_screenshot("here.jpg")
gSearchEle.send_keys(Keys.ENTER)
driver.save_screenshot("here.jpg")

True

In [92]:
driver.save_screenshot("here.jpg")

/usr/local/lib/python3.10/dist-packages/selenium/webdriver/remote/webdriver.py:829: UserWarning: name used for saved screenshot does not match file type. It should end with a `.png` extension
  return self.get_screenshot_as_file(filename)


True

In [ ]:
### 미션!!! 학교 알리미 사이트에 접속 후
### 초등학교 라고 검색
### 관심대상 지역의 초등학생 수를 수집한다!

### 미션2!!! www.timeanddate.com/holidays 접속 후
### 관심대상 나라의 연휴정보 2024년도 기준!!!
### 데이터를 수집한다

### 1. 학교 알리미 홈페이지에 접속한다 (https://www.schoolinfo.go.kr/)

In [95]:
baseUrl = "https://www.schoolinfo.go.kr/"

In [96]:
driver.get(baseUrl)

In [97]:
driver.save_screenshot("here.jpg")

/usr/local/lib/python3.10/dist-packages/selenium/webdriver/remote/webdriver.py:829: UserWarning: name used for saved screenshot does not match file type. It should end with a `.png` extension
  return self.get_screenshot_as_file(filename)


True

### 2. 검색창에 초등학교라고 입력한다

In [98]:
serchInputXpath = '//*[@id="SEARCH_KEYWORD"]'

In [99]:
serchInputEle = driver.find_element(By.XPATH, serchInputXpath)

In [100]:
inputText = "초등학교"

In [101]:
serchInputEle.send_keys(inputText)

In [103]:
serchBtnXpath = '//*[@id="headerSearchForm"]/button'

In [104]:
serchBtnEle = driver.find_element(By.XPATH, serchBtnXpath)
serchBtnEle.click()

In [105]:
driver.save_screenshot("here.jpg")

/usr/local/lib/python3.10/dist-packages/selenium/webdriver/remote/webdriver.py:829: UserWarning: name used for saved screenshot does not match file type. It should end with a `.png` extension
  return self.get_screenshot_as_file(filename)


True

### 3. 검색 버튼에서 설립구분,관심지역을 선택한다

In [107]:
from selenium.webdriver.support.ui import Select

In [106]:
gubunBaseXpath = '//*[@id="FNDN_SC_CD"]'
gubunBaseEle = driver.find_element(By.XPATH, gubunBaseXpath)

In [108]:
selectObj = Select( gubunBaseEle )

In [109]:
gubunBaseText = "설립구분"

In [110]:
selectObj.select_by_visible_text(gubunBaseText)

In [111]:
gubunSidoXpath = '//*[@id="SIDO_CODE"]'

In [112]:
gubunSidoEle = driver.find_element(By.XPATH, gubunSidoXpath)

In [113]:
sidoSelectObj = Select(gubunSidoEle)
gubunSidoText = "서울특별시"
sidoSelectObj.select_by_visible_text(gubunSidoText)

In [117]:
gubunGuXpath = '//*[@id="GUGUN_CODE"]'

In [118]:
gubunGuEle = driver.find_element(By.XPATH, gubunGuXpath)

In [119]:
guSelectObj = Select(gubunGuEle)
gubunGuText = "강서구"
guSelectObj.select_by_visible_text(gubunGuText)

In [120]:
driver.save_screenshot("here.jpg")

/usr/local/lib/python3.10/dist-packages/selenium/webdriver/remote/webdriver.py:829: UserWarning: name used for saved screenshot does not match file type. It should end with a `.png` extension
  return self.get_screenshot_as_file(filename)


True

### 4. 조회버튼을 클릭한다.

In [122]:
selectGuBtnXpath = '//*[@id="contents"]/div/div[1]/div/a'

In [123]:
selectGuBtnEle = driver.find_element(By.XPATH, selectGuBtnXpath)

In [124]:
selectGuBtnEle.click()

In [125]:
driver.save_screenshot("here.jpg")

/usr/local/lib/python3.10/dist-packages/selenium/webdriver/remote/webdriver.py:829: UserWarning: name used for saved screenshot does not match file type. It should end with a `.png` extension
  return self.get_screenshot_as_file(filename)


True

### 더보기 버튼 클릭

In [126]:
moreViewXpath = '//*[@id="contents"]/div/div[2]/a'

In [127]:
moreViewEle = driver.find_element(By.XPATH, moreViewXpath)

In [128]:
moreViewEle.click()

In [129]:
driver.save_screenshot("here.jpg")

/usr/local/lib/python3.10/dist-packages/selenium/webdriver/remote/webdriver.py:829: UserWarning: name used for saved screenshot does not match file type. It should end with a `.png` extension
  return self.get_screenshot_as_file(filename)


True

In [130]:
html = driver.page_source

### 여기까지가 selenium 역할 끝!

In [133]:
bs = bs4.BeautifulSoup( html, "html.parser")
bs

<html class="history no-ie8compat json svg hashchange video csscalc rgba placeholder inlinesvg supports mediaqueries no-touchevents cssvhunit cssvwunit cssanimations backgroundsize boxshadow flexbox csstransforms csstransforms3d csstransitions objectfit object-fit -webkit-" lang="ko" style=""><head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="학교알리미(초·중등 교육정보 공시서비스)" name="title"/>
<meta content="학교알리미_초·중등 교육정보 공시서비스" name="description"/>
<meta content="학교알리미(초·중등 교육정보 공시서비스)" name="keyword"/>
<meta content="학교알리미(초·중등 교육정보 공시서비스)" property="og:title"/>
<meta content="학교알리미_초·중등 교육정보 공시서비스" property="og:description"/>
<meta content="https://www.schoolinfo.go.kr" property="og:url"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>학교알리미(초·중등 교육정보 공시서비스)</title>
<!-- css -->
<style data-href="/css/new/common.css" media="">@charset "utf-8";

/* noto-sans-kr-regular - latin */
/* noto-sans-kr-regular - l